In [ ]:
%%capture
!pip install git+https://github.com/vyperlang/titanoboa.git@master

In [ ]:
from google.colab import userdata
import boa

## **Setting Up the Infrastructure**

Before we can simulate some transactions, we need to initialize the necessary contract. For API keys, we can use the "Secrets" feature of Google Colab. An `RPC_ETHEREUM` including an HTTP key (e.g., from Alchemy) and an `ETHERSCAN_API_KEY` including an Etherscan API key need to be created before the notebook can be used.

Learn more here: https://docs.curve.fi/references/notebooks/


In [ ]:
boa.env.fork(userdata.get('RPC_ETHEREUM'))

CONTROLLER = '0xcad85b7fe52b1939dceebee9bcf0b2a5aa0ce617'
WBTC = '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'

controller = boa.from_etherscan(
    address=CONTROLLER,
    name='CRV Lending Market Controller',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

btc = boa.from_etherscan(
    address=WBTC,
    name='wrapped BTC',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

controller, btc

## **Creating a Loan**

There are two functions to create loans:
- `create_loan` to create a simple loan.
- `create_loan_extended` for creating leveraged loans.

This notebook will only cover the `create_loan` function. The extended function to create leveraged loans will be covered in another notebook.

### **`create_loan`**
`Controller.create_loan(collateral: uint256, debt: uint256, N: uint256):`

Function to create a new loan, requiring specification of the amount of `collateral` to be deposited into `N` bands and the amount of `debt` to be borrowed. The lower the bands chosen, the higher the loss when the position is in soft-liquidation. If there is already an existing loan, the function will revert.

| Input        | Type      | Description                                                                           |
|--------------|-----------|---------------------------------------------------------------------------------------|
| `collateral` | `uint256` | Amount of collateral to use.                                                          |
| `debt`       | `uint256` | Amount of debt to take.                                                               |
| `N`          | `uint256` | Number of bands to deposit into; must range between `MIN_TICKS` and `MAX_TICKS`.      |


In [ ]:
# for this example, we are going to create a loan on the wBTC lending market and borrow some crvUSD

USER = '0x3ee18B2214AFF97000D974cf647E7C347E8fa585'

collateral = 100000000    # amount of collateral tokens (at its native precision)
debt = 30000 * 10**18     # amount of debt to take
N = 10                    # band range: can range between 4 and 50


with boa.env.prank(USER):
  btc.approve(controller, collateral)
  controller.create_loan(collateral, debt, N)

  assert controller.loan_exists(USER) == True

**There are various functions to check information about existing loans:**

- `loan_exists`: Returns a boolean indicating whether a loan for a specific user exists.
- `debt`: Returns the total debt of the loan. This value increases each second due to the charged interest.
- `user_prices`: Returns the upper and lower liquidation prices of the loan. Whenever the price is within this range, the loan is undergoing soft- or de-liquidation.
- `user_state`: Returns collateral amount, stablecoin amount, debt, and number of bands.

*Let's check these methods with our newly created loan:*

In [ ]:
# For simplicity, we return the values normalized by their decimals and round them.
print("Does a loan exist?", controller.loan_exists(USER))
print("Debt:", round(controller.debt(USER) / 1e18, 2))

liq_prices = controller.user_prices(USER)
print(f"Upper liquidation price: {liq_prices[0]} -> {round(liq_prices[0] / 1e18, 2)}")
print(f"Lower liquidation price: {liq_prices[1]} -> {round(liq_prices[1] / 1e18, 2)}")


print("--------")
user_state = controller.user_state(USER)
print("USER STATE:")
print("Collateral:", round(user_state[0] / 1e8, 2))
print("Stablecoin:", round(user_state[1] / 1e18, 2))
print("Debt:", round(user_state[2] / 1e18, 2))
print("Number of bands (N):", user_state[3])

## **Helper Functions**

There are also various "helper functions" which return useful information when creating a loan:

- `max_borrowable`: Returns the maximum borrowable amount when putting up x amount of collateral with n bands.
- `min_collateral`: Returns the minimum collateral needed to support a certain amount of debt with n bands.
- `calculate_debt_n1`: Returns the upper band where collateral is deposited.

In [ ]:
## `max_borrowable`

collateral = 1 * 10**8
N = 10

print(f"Maximum borrowable debt: {controller.max_borrowable(collateral, N)}")

# the borrowable amount is dependant on the number of bands choosen; max bands = 4 and min bands = 50
for i in range(4, 51):
  print(f"Max borrowable with {i} bands: {controller.max_borrowable(collateral, i)}")

*The maximum borrowable amount reduces when increasing the number of bands to deposit into.*

In [ ]:
## `min_collateral`

debt = 30000 * 10**18
N = 10

minimum_collateral = controller.min_collateral(debt, N)

print(f"Amount of collateral required to support the debt: {minimum_collateral} -> {round(minimum_collateral / 1e8, 2)} BTC")

In [ ]:
## `calculate_debt_n1`

collateral = 1 * 10**8
debt = 30000 * 10**18
N = 10

upper_band = controller.calculate_debt_n1(collateral, debt, N)

print("Upper band to deposit into:", upper_band)
print("Lower band to deposit into:", upper_band - N + 1)